In [ ]:
!pwd

In [ ]:
%conda install python-dotenv wandb -y

# Setting lerobot env
from lerobot_util import setup_lerobot_env

setup_lerobot_env()

In [ ]:
base_repo="fanaustinca/fll_v3_batch_"
n_datasets = 2
combined_repo_id = "fanaustinca/fll_v3_combined"

# Create the list of repo IDs: ['repo_1', 'repo_2', ...]
repo_list = [f"{base_repo}{i}" for i in range(1, n_datasets + 1)]
    
# Format the list as a string compatible with the CLI argument parser
# Note: LeRobot often expects the list as a string representation of a Python list
repo_str = str(repo_list).replace(" ", "")
    
command_args = [
    "lerobot-edit-dataset",
    f"--repo_id {combined_repo_id}",
    "--operation.type merge",
    f"--operation.repo_ids \"{repo_str}\"",
    "--push_to_hub true"
]

command = " ".join(command_args)

print(f"Running command: {command}")
!{command}

In [ ]:
from lerobot_util import cd_lerobot, print_shell_md
import json
import sys

cd_lerobot()

# Training ACT model.
# Adjust the following variables before starting training
tag = '100episodes'
policy_type = 'act'

if policy_type == 'diffusion':
    n_action_steps = 10
    chunk_size = 50
elif policy_type == 'act':
    n_action_steps = 50
    chunk_size = 100
else:
    raise ValueError(f"policy_type can only be 'act' or 'diffusion', got: {policy_type}")

hf_namespace = 'fanaustinca'
training_repo = f"{hf_namespace}/fll_v3_combined"
model_repo = f"{hf_namespace}/fll_act_model_100episodes_v3"
# Use 'local_output' consistently
local_output = f"outputs/model/{model_repo}"
steps = 100000
batch_size = 64
num_workers = 12
enable_image_transforms = False

# Clean up output from last run
!rm -rf {local_output}

tfs_config = {
  "to_tensor": {"type": "ToImage"},
  "affine": {
    "type": "RandomAffine",
    "kwargs": {"degrees": 0, "translate": [0.05, 0.05]}
  }
}

# Convert to a compact JSON string for the CLI
tfs_string = json.dumps(tfs_config, separators=(',', ':'))

arguments = [
    "src/lerobot/scripts/lerobot_train.py",
    f"--policy.type={policy_type}",
    f"--dataset.repo_id={training_repo}",
    f"--job_name=fll_training_{policy_type}",
    f"--output_dir={local_output}",
    f"--policy.chunk_size={chunk_size}",
    f"--policy.n_action_steps={n_action_steps}",
    "--policy.device=cuda",
    f"--steps={steps}",
    "--save_freq=5000",
    f"--batch_size={batch_size}",
    "--log_freq=100",
    f"--num_workers={num_workers}",
    "--policy.use_amp=true",
    "--policy.push_to_hub=true",
    f"--policy.repo_id={model_repo}",    
    "--wandb.enable=true"
]

if enable_image_transforms:
    arguments += [
        "--dataset.image_transforms.enable=true",
        "--dataset.image_transforms.random_order=true"
    ]

if policy_type == 'diffusion':
    arguments += ["--policy.num_inference_steps=30"]

# Your variables
print_shell_md("Run the following command to start training:",
               f"{sys.executable}",
               *arguments)

print_shell_md(
    "After training is complete, run the following command to create a tag for the trained model:",
    f"hf repo tag create {model_repo} {tag} -m \"Diffusion model trained on {training_repo}, tag {tag}\"")
